# Importamos librerías y definimos funciones:

In [1]:
import json

from pymongo import MongoClient
from pymongo import GEOSPHERE

from geopy.geocoders import Nominatim


In [18]:
def type_point(lista):
    return {"type":"Point", "coordinates": lista}

def consigue_lat_long(direccion):
    """
    Esta función saca las coordenadas de la dirección que le pases
    y la devuelve como lista [lat,long]
    """
    geolocator = Nominatim(user_agent="dddddddddddddddddd@gmail.com")
    location = geolocator.geocode(direccion)
    return [location.latitude, location.longitude]

def consigue_type_point_lat_long(direccion):
    """
    Esta función saca las coordenadas de la dirección que le pases
    y la devuelve en type:point 
    """
    return type_point( consigue_lat_long(direccion) )

def permutar_lat_x_long(lugar):
    lugar_T = type_point([lugar["coordinates"][1] , lugar["coordinates"][0] ])
    return lugar_T

def buscar_barrio(lugar) :
    query = {"geometry": {"$geoIntersects": {"$geometry": lugar}}}
    c    = db.get_collection("Barrios")
    proy = {"_id":0, "properties.DISPLAY_NAME": 1, "properties.MOVEMENT_ID": 1}
    try:
        return c.find_one(query,proy)["properties"]['MOVEMENT_ID']
    except:
        return False

def buscar_codigo_postal(lugar) :
    query = {"geometry": {"$geoIntersects": {"$geometry": lugar}}}
    c    = db.get_collection("Codigos_Postales")
    proy = {"_id":0, "properties.GEOCODIGO": 1   , "properties.MOVEMENT_ID": 1}
    try:
        return c.find_one(query,proy)["properties"]['MOVEMENT_ID']
    except:
        return False


# Cargamos los geojson en este Notebook

In [3]:
geo_barrios = json.load( open ("data/madrid_barrios.geojson")          )
geo_codpost = json.load( open ("data/madrid_codigos_postales.geojson") )

In [4]:
# Exportamos los geojson a jsons capaces de importar en MongoDB
json.dump(geo_barrios["features"], open ("data/madrid_barrios.json", "w") )
json.dump(geo_codpost["features"], open ("data/madrid_codigos_postales.json", "w") )

# Conectamos con la base de datos en MongoDB

In [5]:
# Conectamos con la base de datos:

client = MongoClient("localhost:27017")
db = client.get_database("Coropleticos_Madrid")

In [6]:
# Creamos la indexación geoespacial de MongoDB para ejecutar eficientemente consultas espaciales
# Esto sólo debe hacerse una vez, y luego la colección de Mongo está correctamente indexada para siempre.

#db.Barrios.create_index([("geometry", GEOSPHERE)])
#db.Codigos_postales.create_index([("geometry", GEOSPHERE)])


In [26]:
# Cargamos las colecciones desde MongoDB a este Notebook
codigos_postales_mongo    = db.get_collection("Codigos_Postales")
# Compruebo que la conexion funciona (con codigos postales)
codigos_postales_mongo.find_one().keys()

dict_keys(['_id', 'type', 'properties', 'geometry'])

In [27]:
codigos_postales_mongo.find_one()["properties"]

{'CODBDT': 828913,
 'GEOCODIGO': '28001',
 'MOVEMENT_ID': '1',
 'DISPLAY_NAME': 'Código Postal 28001'}

In [28]:
# Cargamos las colecciones desde MongoDB a este Notebook
barrios_mongo             = db.get_collection("Barrios")
# Compruebo que la conexion funciona (con barrios)
barrios_mongo.find_one().keys()

dict_keys(['_id', 'type', 'properties', 'geometry'])

In [29]:
barrios_mongo.find_one()["properties"]

{'CODBDT': 860645,
 'GEOCODIGO': '079011',
 'MOVEMENT_ID': '1',
 'DISPLAY_NAME': '011 Palacio'}

# Definimos lugares a buscar

In [94]:
origen = "Paseo de la chopera, 14, Madrid"
origen_ll   = consigue_type_point_lat_long(origen)
print(origen_ll)
origen_ll_T = permutar_lat_x_long(origen_ll)
print(origen_ll_T)

{'type': 'Point', 'coordinates': [40.3920606, -3.6978319]}
{'type': 'Point', 'coordinates': [-3.6978319, 40.3920606]}


In [95]:
destino = "Puerta del Sol, Madrid"
destino_ll   = consigue_type_point_lat_long(destino)
print(destino_ll)
destino_ll_T = permutar_lat_x_long(destino_ll)
print(destino_ll_T)

{'type': 'Point', 'coordinates': [40.4168624, -3.7043032918742496]}
{'type': 'Point', 'coordinates': [-3.7043032918742496, 40.4168624]}


In [98]:
origen_ll["coordinates"]

[40.3920606, -3.6978319]

In [100]:
destino_ll["coordinates"]

[40.4168624, -3.7043032918742496]

In [103]:
punto_central = [ (origen_ll["coordinates"][0]+destino_ll["coordinates"][0])/2 , 
                  (origen_ll["coordinates"][1]+destino_ll["coordinates"][1])/2 ]
type_point(punto_central)

{'type': 'Point', 'coordinates': [40.4044615, -3.7010675959371246]}

# Preparamos las geoqueries y la proyeccion con los datos interesantes

In [ ]:
# lugar tiene que ir como tipo Point, con coordenadas al reves
query = {"geometry": {"$geoIntersects": {"$geometry": lugar}}}

In [ ]:
# Me guardo la proyección para que solo me dé el nombre del distrito
barrios_proy          = {"_id":0, "properties.DISPLAY_NAME": 1, "properties.MOVEMENT_ID": 1}
codigos_postales_proy = {"_id":0, "properties.GEOCODIGO": 1   , "properties.MOVEMENT_ID": 1}

# Busco en la base de datos con la query y proyeccion

In [19]:
b = buscar_barrio(destino)
b

False

In [20]:
b = buscar_barrio(destino_ll)
b

False

In [21]:
b = buscar_barrio(destino_ll_T)
b

'6'

In [23]:
cp = buscar_codigo_postal(destino)
cp

False

In [24]:
cp = buscar_codigo_postal(destino_ll)
cp

False

In [25]:
cp = buscar_codigo_postal(destino_ll_T)
cp

'13'

# Me hago un GeoJson con los barrios origen y destino

In [38]:
geo_barrios = json.load( open ("data/madrid_barrios.geojson")          )
geo_codpost = json.load( open ("data/madrid_codigos_postales.geojson") )

In [86]:
origin = "Puerta del Sol, Madrid"
origin_ll = consigue_lat_long(origin)
origin_ll_T = permutar_lat_x_long(consigue_type_point_lat_long(origin_ll))

geo_Reduced = {'type': 'FeatureCollection',
               'features': []}

search_by = {}
search_by["origin"] = ( "barrio" , buscar_barrio(origin_ll_T) )
if search_by["origin"][1] != False:
    query = {"geometry": {"$geoIntersects": {"$geometry": origin_ll_T}}}
    c    = db.get_collection("Barrios")
    origin_geo = c.find_one(query)
    origin_geo.pop('_id')
    geo_Reduced["features"].append(origin_geo)
    geo_Reduced["features"].append(origin_geo)
elif search_by["origin"][1] == False:
    search_by["origin"][0] = ( "codigo_postal" , buscar_codigo_postal(origin_ll_T) )
    if search_by["origin"][1] == False:
        search_by["origin"] = ( "No encontrado" , "No encontrado" )
    
destiny = "Paseo de la Chopera, 14 Madrid"
destiny_ll = consigue_lat_long(destiny)
destiny_ll_T = permutar_lat_x_long(consigue_type_point_lat_long(destiny_ll))
search_by["destiny"] = ( "barrio" , buscar_barrio(destiny_ll_T) )
if search_by["destiny"][1] != False:
    query = {"geometry": {"$geoIntersects": {"$geometry": destiny_ll_T}}}
    c    = db.get_collection("Barrios")
    destiny_geo = c.find_one(query)
    destiny_geo.pop('_id')
    geo_Reduced["features"].append(destiny_geo)
elif search_by["destiny"][1] == False:
    search_by = ( "codigo_postal" , buscar_codigo_postal(destiny_ll_T) )
    if search_by["destiny"][1] == False:
        search_by["destiny"] = ( "No encontrado" , "No encontrado" )
print(search_by)

{'origin': ('barrio', '6'), 'destiny': ('barrio', '9')}


In [104]:
#geo_Reduced

In [88]:
geo_Reduced["type"]

'FeatureCollection'

In [90]:
len(geo_Reduced["features"])

2

In [93]:
geo_Reduced["features"][0]["properties"][""]

{'CODBDT': 860650,
 'GEOCODIGO': '079016',
 'MOVEMENT_ID': '6',
 'DISPLAY_NAME': '016 Sol'}

In [109]:
import folium

punto_central = [(origin_ll[0]+destiny_ll[0])/2 , (origin_ll[1]+destiny_ll[1])/2 ]

map_1 = folium.Map(location = punto_central, zoom_start=13)

folium.GeoJson(geo_Reduced, style_function=lambda feature: {'fillColor': 'blue',
                                                            'color': 'blue',
                                                            'weight': 1,
                                                            'dashArray': '5, 5'} ).add_to(map_1)

map_1